# Import Libraries 

In [2]:
import findspark                                      #Import library to Search for Spark Installation  

findspark.init()                                      #Search Spark Installation

from pyspark.sql import SparkSession                  # Import of Spark Session
from pyspark.sql import Row
from pyspark import SparkContext as spark             # Import the Regular Spark Contex
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()
import threading
import os

sc = spark.sparkContext                               #Initialize Spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1575630339505_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----------

# Creating Variables

## Buckets

In [ ]:
SPG_MANUAL_INPUT_BUCKET = os.environ['MANUAL_INPUT_BUCKET']

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

In [ ]:
GERDAU_BUCKET = os.environ['GERDAU_BUCKET']

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

## Input Paths

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET_IBGE = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_IBGE/SPG_RG_IBGE.parquet"

In [ ]:
SPG_MANUAL_INPUT_BUCKET_BRANCH = "INPUT/tb_SPG_SUPPORT_CIDADE_FILIAL.csv"

In [ ]:
GERDAU_BUCKET_JURISDICTION_INPUT_PATH = "global/masterdata/j1btxjurt"

## Output Paths

In [ ]:
SPG_OUTPUT_BUCKET_REGION = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_REGIOES/SPG_RG_REGIOES.parquet"

-------------

# Creating Defined

In [9]:
def remove_some_hifen(col_name):
    removed_chars = ("-")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
def remove_some_accents(col_name):
    removed_array = [('Á', 'A'), ('Ã', 'A'),('À', 'A'),('Â', 'A'),('Ä', 'A'),
                    ('É', 'E'),('È', 'E'),('Ê', 'E'),('Ë', 'E'),
                    ('Í', 'I'),('Ì', 'I'),('Î', 'I'),('Ï', 'I'),
                    ('Ó', 'O'),('Õ', 'O'), ('Ò', 'O'),('Ô', 'O'),('Ö', 'O'),
                    ('Ú', 'U'),('Ù', 'U'),('Û', 'U'),('Ü', 'U'),
                    ('Ç', 'C')]
    r = col_name
    for a, b in removed_array:
        r = regexp_replace(r, a, b)
    return r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def remove_some_space(col_name):
    removed_chars = (" ")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def remove_some_apostrophe(col_name):
    removed_chars = ("'")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Read PARQUET with IBGE Cities (SPG_RG_IBGE)

In [3]:
spark_df_IBGE_cities = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_IBGE)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----------

# Read CVS with Branches (SPG_RG_REGIOES)

In [4]:
# Reading temporary CSV file from the SandBox Data Lake
df_branch = spark.read.format("csv").option("header","true").option("sep",";").option("encoding", "ISO-8859-1").load("s3a://"+SPG_MANUAL_INPUT_BUCKET+"/"+SPG_MANUAL_INPUT_BUCKET_BRANCH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

------------

# Read CSV with SAP Cities (j_1btxjur)

In [5]:
CSV_SAP_cities = spark.read.parquet("s3a://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_JURISDICTION_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
CSV_SAP_cities=CSV_SAP_cities.filter(CSV_SAP_cities.SPRAS.like("%P%")&
                                    CSV_SAP_cities.COUNTRY.like("%BR%"))\
                             .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
CSV_SAP_cities=CSV_SAP_cities.select(CSV_SAP_cities.COUNTRY, CSV_SAP_cities.TAXJURCODE, CSV_SAP_cities.TEXT)\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Selecting Necessary Columns from Branch Table 
df_branch=df_branch.select('Município corrigido', 'Filial SUGESTÃO', 'UF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Get SAPs IBGE Code

In [14]:
CSV_SAP_cities = CSV_SAP_cities.withColumn('RG_COD_IBGE', substring(CSV_SAP_cities["TAXJURCODE"], -7, 7))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
CSV_SAP_cities = CSV_SAP_cities.withColumn('RG_REG_IBGE', substring(CSV_SAP_cities["TAXJURCODE"], -10, 3))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
for col in CSV_SAP_cities.columns:
    CSV_SAP_cities = CSV_SAP_cities.withColumn(col, ltrim(rtrim(CSV_SAP_cities[col])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
CSV_SAP_cities=CSV_SAP_cities.dropDuplicates(["RG_COD_IBGE","RG_REG_IBGE"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Remove Invalid Row

In [18]:
CSV_SAP_cities = CSV_SAP_cities.where(CSV_SAP_cities.TAXJURCODE != "TAXJURCODE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Change the Columns Name

In [19]:
CSV_SAP_cities = CSV_SAP_cities.select(CSV_SAP_cities.TAXJURCODE.alias("SAP_COD") 
                                       ,CSV_SAP_cities.TEXT.alias("SAP_NAME")
                                       ,CSV_SAP_cities.RG_COD_IBGE.alias("RG_COD_SAP_IBGE")
                                       ,CSV_SAP_cities.RG_REG_IBGE.alias("RG_REG_IBGE")
                                       ,CSV_SAP_cities.COUNTRY.alias("COUNTRY"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Separating between ZF and Regions

In [20]:
SAP_cities_Region = CSV_SAP_cities.filter("RG_REG_IBGE like 'ZF%'")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
SAP_cities_Region = SAP_cities_Region.select(SAP_cities_Region.RG_COD_SAP_IBGE, SAP_cities_Region.RG_REG_IBGE.alias("RG_ZN_SAP"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
CSV_SAP_cities = CSV_SAP_cities.filter("RG_REG_IBGE not like 'ZF%'")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Populating ZF column

In [23]:
SAP_cities_Region = SAP_cities_Region.withColumn('CITY_ZF', lit(1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
SAP_cities_Region=SAP_cities_Region.dropDuplicates(["RG_COD_SAP_IBGE"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Left join between SAP_cities_Region and CSV_SAP_cities

In [25]:
CSV_SAP_cities = CSV_SAP_cities.join(SAP_cities_Region, CSV_SAP_cities.RG_COD_SAP_IBGE == SAP_cities_Region.RG_COD_SAP_IBGE,how='left')\
                               .withColumn("RG_COD_IBGE",coalesce(SAP_cities_Region.RG_COD_SAP_IBGE,CSV_SAP_cities.RG_COD_SAP_IBGE))\
                               .withColumn("RG_REG_SAP",coalesce(CSV_SAP_cities.RG_REG_IBGE, SAP_cities_Region.RG_ZN_SAP))\
                               .drop("RG_COD_SAP_IBGE"
                                     ,"RG_REG_IBGE"
                                     ,"RG_ZN_SAP")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
CSV_SAP_cities= CSV_SAP_cities.fillna({'CITY_ZF':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
CSV_SAP_cities=CSV_SAP_cities.dropDuplicates(["RG_COD_IBGE"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Full Join IBGE - SAP -> PK = ID

In [28]:
spark_df_cities = spark_df_IBGE_cities.join(CSV_SAP_cities,spark_df_IBGE_cities.IBGE_ID.cast("int") == CSV_SAP_cities.RG_COD_IBGE.cast("int"),how='full')\
                                      .withColumn("IBGE_SAP_CITY_ID", coalesce(spark_df_IBGE_cities.IBGE_ID, CSV_SAP_cities.RG_COD_IBGE))\
                                      .withColumn("IBGE_SAP_CITY_NAME",coalesce(spark_df_IBGE_cities.IBGE_NAME,CSV_SAP_cities.SAP_NAME))\
                                      .withColumn("IBGE_UF_ACRONYMS",coalesce(spark_df_IBGE_cities.IBGE_UF_ACRONYMS,CSV_SAP_cities.RG_REG_SAP))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
spark_df_cities=spark_df_cities.drop("__index_level_0__")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
spark_df_cities=spark_df_cities.dropDuplicates(["IBGE_SAP_CITY_ID"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Change Columns Name

In [37]:
spark_df_cities=spark_df_cities.withColumnRenamed("IBGE_NAME", "IBGE_CITY_NAME")\
                               .withColumnRenamed("IBGE_ID", "IBGE_CITY_ID")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
spark_df_cities=spark_df_cities.withColumn("SAP_NAME", upper(spark_df_cities.SAP_NAME))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df_branch=df_branch.withColumn("UPPER_CITY", upper(remove_some_accents(df_branch["Município corrigido"])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
spark_df_cities = spark_df_cities.join(df_branch, (upper(remove_some_accents(remove_some_apostrophe(remove_some_space(remove_some_hifen(spark_df_cities["IBGE_CITY_NAME"]))))) == remove_some_apostrophe(remove_some_space(remove_some_hifen(df_branch["UPPER_CITY"]))))&
                                                  (upper(trim(spark_df_cities["IBGE_UF_ACRONYMS"])) == upper(trim(df_branch["UF"]))), how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Renaming and dropping columns from Branch Table
spark_df_cities = spark_df_cities.withColumnRenamed("Filial SUGESTÃO", "BRANCH1")\
                                 .drop("Município corrigido")\
                                 .drop("UPPER_CITY")\
                                 .drop("UF")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
spark_df_cities = spark_df_cities.join(df_branch, (upper(remove_some_apostrophe(remove_some_space(remove_some_hifen(spark_df_cities["SAP_NAME"])))) == remove_some_apostrophe(remove_some_space(remove_some_hifen(df_branch["UPPER_CITY"]))))&
                                                  (upper(trim(spark_df_cities["IBGE_UF_ACRONYMS"])) == upper(trim(df_branch["UF"]))), how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# Renaming and dropping columns from Branch Table
spark_df_cities = spark_df_cities.withColumnRenamed("Filial SUGESTÃO", "BRANCH2")\
                                 .drop("Município corrigido")\
                                 .drop("UPPER_CITY")\
                                 .drop("UF")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
spark_df_cities = spark_df_cities.withColumn("BRANCH", coalesce(spark_df_cities.BRANCH1, spark_df_cities.BRANCH2))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
spark_df_cities = spark_df_cities.drop("BRANCH1", "BRANCH2")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Upload Parquet File to AWS S3

## Upload Final Table

In [46]:
spark_df_cities=spark_df_cities.distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
spark_df_cities=spark_df_cities.withColumn('IBGE_SAP_CITY_NAME', upper(remove_some_accents(upper(spark_df_cities.IBGE_SAP_CITY_NAME))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
spark_df_cities = spark_df_cities.repartition(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
spark_df_cities.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_REGION, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…